## Retrieve house and senate speech json files

In [1]:
import json
with open('data/senate_speech (1).json') as json_file:
    senate_speech_1 = json.load(json_file)

In [2]:
with open('data/senate_speech (2).json') as json_file:
    senate_speech_2 = json.load(json_file)

In [3]:
with open('data/house_speech.json') as json_file:
    house_speech = json.load(json_file)

## Merge all three files into a dictionary

In [4]:
def Merge(dict_1, dict_2):
    result = dict_1 | dict_2
    return result

senate_speech = Merge(senate_speech_1, senate_speech_2)
speech = Merge(senate_speech, house_speech)

In [5]:
print(speech.keys())

dict_keys(['Mr. GRASSLEY', 'Mr. SCHUMER', 'Mr. CORNYN', 'Mr. HAWLEY', 'Mr. KAINE', 'Mr. DURBIN', 'Mr. CARDIN', 'Mr. THUNE', 'Mr. BROWN', 'Mr. BLUMENTHAL', 'Mr. GARDNER', 'Ms. COLLINS', 'Mrs. BLACKBURN', 'Mr. MURPHY', 'Mr. BLUNT', 'Mr. RUBIO', 'Mr. ROBERTS', 'Mr. MENENDEZ', 'Mr. BARRASSO', 'Mr. MARKEY', 'Mr. INHOFE', 'Mr. BOOZMAN', 'Mr. WHITEHOUSE', 'Ms. HARRIS', 'Mr. RISCH', 'Mr. TOOMEY', 'Ms. DUCKWORTH', 'Mrs. FISCHER', 'Mr. UDALL', 'Mr. PORTMAN', 'Mrs. MURRAY', 'Mr. COTTON', 'Ms. SMITH', 'Mr. LANKFORD', 'Ms. BALDWIN', 'Mr. DAINES', 'Ms. WARREN', 'Mr. BOOKER', 'Ms. STABENOW', 'Mr. PETERS', 'Mr. JOHNSON', 'Mr. LEAHY', 'Mr. CARPER', 'Mr. MERKLEY', 'Mr. TESTER', 'Mr. WYDEN', 'Mr. SCOTT of Florida', 'Mr. CRAMER', 'Mr. HOEVEN', 'Mr. SULLIVAN', 'Ms. McSALLY', 'Mr. CRAPO', 'Mr. CASEY', 'Mr. REED', 'Mr. ENZI', 'Mr. WICKER', 'Ms. CANTWELL', 'Mr. SANDERS', 'Mr. TILLIS', 'Ms. MURKOWSKI', 'Mrs. SHAHEEN', 'Mr. YOUNG', 'Mr. MORAN', 'Mr. CRAWLEY', 'Mrs. FEINSTEIN', 'Ms. HASSAN', 'Mr. SCHIFF', 'Ms. W

## Merge speech data with LES data

In [6]:
import pandas as pd
import fuzzy_pandas as fpd

house = pd.read_excel("data/House116.xlsx")
senate = pd.read_excel("data/Senate116.xlsx")
house = house[['Legislator name', 'Legislative Effectiveness Score', 'party']]
senate = senate[['Legislator name',
                 'Legislative Effectiveness Score', 'party']]
df_les = pd.concat([house, senate], axis=0)

In [7]:
speech = pd.DataFrame.from_dict(speech, orient="index")
speech.reset_index(inplace=True)
speech.rename(columns = {'index': 'name', 0: 'speech'}, 
            inplace = True)
speech['name'] = speech['name'].map(lambda x: x.split(' ')[1].lower())
speech

,name,speech
0,grassley,Mr. GRASSLEY. I do agree. The IRS got the law...
1,schumer,"Mr. SCHUMER. Mr. President, last night, the H..."
2,cornyn,"Mr. CORNYN. Mr. President, following the long..."
3,hawley,"Mr. HAWLEY. Madam President, just two brief p..."
4,kaine,"Mr. KAINE. Mr. President, today I am introduc..."
...,...,...
609,mcbath,"Mrs. MCBATH. Madam Speaker, I move to suspen..."
610,mcbath,"Ms. McBATH. Madam Speaker, I urge my colleagu..."
611,lee,"Mr. LEE of California. Mr. Speaker, let me al..."
612,jackson-lee,Ms. JACKSON-LEE. Then I would like to emphasi...


## Record linkage by extracting legislator's last name in lowercase

In [8]:
df_les['Name'] = df_les['Legislator name'].map(lambda x: x.split(' ')[1].lower())

In [9]:
def record_link(df1, df2, threshold, name):
    """
    Merge a dictionary with the LES dataset using record linkage

    Inputs:
        dic: dictionary object
        df2: dataframe containing the LES dataset
        threshold: threshold for record linkage
        name: name of the output csv file
    Outputs:
        /
    """
    df_analysis = fpd.fuzzy_merge(df1, df2,
                                  left_on='Name',
                                  right_on='name',
                                  method='levenshtein',
                                  threshold=threshold)
    df_analysis.to_csv("data/"+name+".csv")


In [10]:
record_link(df_les, speech, 1, "speech_data")

## Write all rows to the same csv file

In [11]:
raw_issue_analysis = pd.read_csv("data/speech_data.csv")

In [12]:
raw_issue_analysis = raw_issue_analysis[['Legislator name', 'Legislative Effectiveness Score', 'party','speech']]

In [13]:
raw_issue_analysis

,Legislator name,Legislative Effectiveness Score,party,speech
0,Ralph Abraham,0.039761,R,"Mr. ABRAHAM. Mr. Speaker, on Friday, July 24,..."
1,Alma Adams,1.783789,D,"Ms. ADAMS. Mr. Speaker, on that I demand the ..."
2,Robert Aderholt,0.019880,R,"Mr. ADERHOLT. Mr. Speaker, I would like to le..."
3,Pete Aguilar,0.049701,D,"Mr. AGUILAR. Madam Speaker, I thank the gentl..."
4,Rick Allen,0.039761,R,"Mr. ALLEN. Mr. Speaker, I was unable to vote ..."
...,...,...,...,...
714,Elizabeth Warren,0.725357,D,"Ms. WARREN. Mr. President, I ask for the yeas..."
715,Sheldon Whitehouse,0.648167,D,"Mr. WHITEHOUSE. Madam President, we are rapid..."
716,Roger Wicker,3.558171,R,"Mr. WICKER. Madam President, the Senator, my ..."
717,Ron Wyden,0.927056,D,"Mr. WYDEN. Thank you for that background, Mr...."


## Write each row as a separate csv titled by legislator name

In [14]:
import csv
for i, row in raw_issue_analysis.iterrows():
    if i > len(raw_issue_analysis):
        break
    else:
        f = open(f"data/speech_data/{row['Legislator name']}.csv", 'w')
        writer = csv.writer(f)

        # write each row to separate csv file with header
        writer.writerow(['name', 'LES', 'party', 'speech'])
        writer.writerow(row)

        # close the file and increment index
        f.close()
        i+=1